In [ ]:
# 1. Proje Kurulumu ve Güncelleme (Colab stok + eksikler)
import os
import sys
import subprocess
import time

try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Google Colab ortamı algılandı.")
    repo_url = "https://github.com/onndd/newproje.git"
    project_dir = "/content/newproje"

    if os.path.exists(project_dir):
        %cd {project_dir}
        !git pull
    else:
        !git clone {repo_url}
        %cd {project_dir}

    print("Gerekli kütüphaneler yükleniyor...")

    # AGGRESSIVE FIX: Clean old numpy/pandas first
    print("Cleaning conflicting libraries...")
    try:
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy", "pandas"], check=False)
    except Exception as e:
        print(f"Uninstall notice (safe to ignore): {e}")

    print("Installing compatible numpy and pandas...")
    # Force reinstall with specific versions
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2.0", "pandas==2.2.2", "--force-reinstall", "--no-deps"])
    
    # Re-install dependencies needed that might have been removed or need check
    packages = [
        "catboost==1.2.8",
        "lightgbm==4.6.0",
        "hmmlearn==0.3.2",
        "optuna==3.5.0",
        "streamlit==1.29.0",
        "stable-baselines3==2.2.1",
        "shimmy==1.3.0",
        "gymnasium==0.29.1"
    ]
    
    for pkg in packages:
        print(f"Installing {pkg}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
            print(f"Successfully installed {pkg}")
        except subprocess.CalledProcessError as e:
            print(f"❌ ERROR installing {pkg}: {e}")

    if project_dir not in sys.path:
        sys.path.append(project_dir)
    
    print("\n✅ KURULUM TAMAMLANDI.")
    print("🛑🛑 ÇOK ÖNEMLİ: ŞİMDİ 'RUNTIME > RESTART SESSION' YAPMALISINIZ 🛑🛑")
    print("     (Aksi takdirde 'numpy.dtype size changed' hatası gitmeyecektir.)")
else:
    print("Local environment.")
    project_dir = os.getcwd()
    if project_dir not in sys.path:
        sys.path.append(project_dir)


In [ ]:
# 2. Veri Yükleme ve 3'lü Ayrım (3-Way Split)
import sys
import os
if '/content/newproje' not in sys.path:
    sys.path.append('/content/newproje')
import sqlite3
import pandas as pd
import numpy as np
from jetx_project.data_loader import load_data
from jetx_project.config import DB_PATH, PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

print("Veri yükleniyor...")
try:
    df = load_data(DB_PATH)
    print(f"Toplam {len(df)} veri yüklendi.")
    
    # --- 3-WAY SPLIT STRATEJİSİ ---
    # 1. Base Train (%70): Temel modelleri eğitmek için.
    # 2. Meta Train (%15): Temel modellerin tahminlerini alıp Meta-Learner'ı eğitmek için.
    # 3. Test (%15): Final simülasyon ve değerlendirme için (Hiçbir model burayı görmez).
    
    total_len = len(df)
    base_train_end = int(total_len * 0.70)
    meta_train_end = int(total_len * 0.85)
    
    print(f"--- Veri Ayrımı ---")
    print(f"Base Train: 0 - {base_train_end} ({base_train_end} örnek)")
    print(f"Meta Train: {base_train_end} - {meta_train_end} ({meta_train_end - base_train_end} örnek)")
    print(f"Test (Sim): {meta_train_end} - {total_len} ({total_len - meta_train_end} örnek)")
    
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen jetx.db dosyasının proje klasöründe olduğundan emin olun.")

In [ ]:
# 2.5. Anomaly Detector (The Shield) Eğitimi
from jetx_project.model_anomaly import train_anomaly_detector, save_anomaly_detector

if 'df' in locals() and len(df) > 500:
    print("Anomaly Detector (The Shield) eğitiliyor...")
    # Base Train verisiyle eğit
    train_values_anomaly = df['value'].values[:base_train_end]
    anomaly_model = train_anomaly_detector(train_values_anomaly)
    save_anomaly_detector(anomaly_model, output_dir='models')
    print("Anomaly Detector tamamlandı.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 3. Psikolojik Modeller (HMM) Eğitimi
print("Psikolojik Modeller (HMM) eğitiliyor...")

# Fallback: If hmmlearn is missing, install it here.
try:
    from hmmlearn import hmm
except ImportError:
    print("⚠️ hmmlearn bulunamadı, şimdi yükleniyor...")
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "hmmlearn==0.3.2"])
    print("hmmlearn yüklendi.")

from jetx_project.model_hmm import train_categorical_hmm, predict_categorical_hmm_states_causal, save_hmm_model

if 'df' in locals() and len(df) > 0:
    # Sadece Base Train verisiyle eğit
    train_values = df['value'].values[:base_train_end]
    
    # HMM Eğitimi (Categorical)
    hmm_model, hmm_map, hmm_bins = train_categorical_hmm(train_values)
    save_hmm_model(hmm_model, hmm_map, bins=hmm_bins, output_dir='models')
    
    # Tüm veri seti için durum tahmini (Feature olarak kullanılacak)
    # CAUSAL PREDICTION: Geleceği görmeden, kayan pencere ile tahmin et.
    all_values = df['value'].values
    hmm_states_mapped = predict_categorical_hmm_states_causal(hmm_model, all_values, hmm_map, bins=hmm_bins, window_size=200)
    
    print("HMM Durumları (Causal) tüm veri için belirlendi.")
    print(f"Durum Dağılımı: {np.bincount(hmm_states_mapped)}")
else:
    print("Veri yok, HMM eğitilemedi.")

## 🐯 MODEL A: CatBoost (The Hunter)

In [ ]:
# 5.0. Model A (CatBoost) - ÖZELLİK ÇIKARIMI 📊
# BU HÜCRE ÇOK ÖNEMLİDİR. X_a (Eğitim Verisi) Burada Oluşur. 
# Çalıştırmazsanız sonraki adımlar "X_a not found" hatası verir.

from jetx_project.model_a import prepare_model_a_data
import pandas as pd
import numpy as np

if 'df' in locals():
    print("🐯 Özellikler Çıkarılıyor (Feature Extraction)...")
    
    # HMM Stateleri (Eğer HMM çalıştıysa, yoksa None)
    # HMM hücresinden 'hmm_states' gelmeli.
    # Güvenlik için kontrol:

    # --- MODIFIED HMM BRIDGE ---
    # Global namespace'ten HMM sonuçlarını çekiyoruz
    if 'hmm_states_causal' in globals():
        hmm_states_mapped = hmm_states_causal
        print("✅ HMM Durumları (causal) başarıyla yüklendi.")
    elif 'hmm_states' in globals():
        hmm_states_mapped = hmm_states
        print("✅ HMM Durumları (standart) başarıyla yüklendi.")
    else:
        print("⚠️ HMM Stateleri bulunamadı, varsayılan (0) kullanılacak.")
        hmm_states_mapped = np.zeros(len(df))
    # ---------------------------

        
    # Veri Hazırlığı
    # X_a: Özellik Matrisi
    # y_p15_a: Hedef (1.50x üstü mü?)
    # y_p3_a: Hedef (3.00x üstü mü?)
    # y_x_a: Hedef (Crash değeri)
    
    X_a, y_p15_a, y_p3_a, y_x_a = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=50)
    
    # Veri Sınırlarını Belirle (Optimization ve Training için)
    # Train / Test ayrımı zaten indekslerle yapılacak ama X_a tüm seti kapsar.
    
    print(f"✅ X_a Oluşturuldu. Boyut: {X_a.shape}")
    print(f"   Örnek Özellikler: {list(X_a.columns[:5])}...")
else:
    print("❌ 'df' bulunamadı. Lütfen 2. Hücreyi (Veri Yükleme) çalıştırın!")


In [ ]:
# 5.1. Model A (CatBoost) - OPTİMİZASYON 🛠️
from jetx_project.optimization import optimize_catboost
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

if 'X_a' in locals():
    print("🐯 Kaplan (CatBoost) için en iyi avlanma ayarları aranıyor...")
    
    # 1. P1.5 (Güvenli Av)
    print("   -> P1.5 Optimizasyonu...")
    best_params_catboost_p15 = optimize_catboost(X_a, y_p15_a, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS)
    
    # 2. P3.0 (Büyük Av)
    print("   -> P3.0 Optimizasyonu...")
    best_params_catboost_p3 = optimize_catboost(X_a, y_p3_a, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS_P3)
    
    print(f"✅ CatBoost Ayarları Bulundu.")
else:
    print("❌ Veri (X_a) bulunamadı. Lütfen Veri Yükleme hücresini çalıştırın.")


In [ ]:
# 5.2. Model A (CatBoost) - EĞİTİM 🏃
from jetx_project.model_a import train_model_a, save_models
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

if 'best_params_catboost_p15' not in locals(): best_params_catboost_p15 = None
if 'best_params_catboost_p3' not in locals(): best_params_catboost_p3 = None

if 'X_a' in locals():
    print("🐯 Kaplan (CatBoost) Eğitiliyor...")
    
    modelA_p15, modelA_p3, modelA_x = train_model_a(
        X_a, y_p15_a, y_p3_a, y_x_a, 
        params_p15=best_params_catboost_p15, 
        params_p3=best_params_catboost_p3,
        scoring_params_p15=PROFIT_SCORING_WEIGHTS, 
        scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3
    )
    
    save_models(modelA_p15, modelA_p3, modelA_x, output_dir='models')
    print("✅ Model A Tamamlandı ve Kaydedildi.")
else:
    print("❌ Veri (X_a) bulunamadı.")


## 👯 MODEL B: k-NN (The Mirror)

In [ ]:
# 6. Model B (k-NN) - HAFIZA OLUŞTURMA 🧠
from jetx_project.model_b import build_memory, train_model_b, save_memory

if 'df' in locals() and len(df) > 500:
    print("👯 İkizler (k-NN) Hafızası Oluşturuluyor...")
    
    train_values_b = df['value'].values[:base_train_end]
    patterns, targets = build_memory(train_values_b)
    nbrs, pca = train_model_b(patterns)
    
    save_memory(nbrs, pca, patterns, targets, output_dir='models')
    print("✅ Model B Hafızası Kaydedildi.")
else:
    print("❌ Yeterli veri yok.")


## 🔮 MODEL C: LSTM (The Oracle)

In [ ]:
# 7.1. Model C (LSTM) - OPTİMİZASYON 🛠️
from jetx_project.optimization import optimize_lstm
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3
import numpy as np

if 'df' in locals():
    print("🔮 Kahin (LSTM) için zaman dalgaları analiz ediliyor (Optuna)...")
    
    train_values_base = df['value'].values[:base_train_end]
    
    # 1. P1.5 (Standard 1.50x Threshold)
    print("   -> P1.5 için Derinlik Ayarlanıyor...")
    best_params_lstm_p15 = optimize_lstm(train_values_base, n_trials=10, scoring_params=PROFIT_SCORING_WEIGHTS, target_threshold=1.50)
    
    # 2. P3.0 (High 3.00x Threshold)
    print("   -> P3.0 için Derinlik Ayarlanıyor (THRESHOLD=3.00)...")
    best_params_lstm_p3 = optimize_lstm(train_values_base, n_trials=10, scoring_params=PROFIT_SCORING_WEIGHTS_P3, target_threshold=3.00)

    print(f"✅ LSTM Ayarları Bulundu.")
else:
    print("❌ Veri bulunamadı.")


In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
# 7.2. Model C (LSTM) - EĞİTİM 🏃
from jetx_project.model_lstm import train_model_lstm, save_lstm_models
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

if 'best_params_lstm_p15' not in locals(): best_params_lstm_p15 = None
if 'best_params_lstm_p3' not in locals(): best_params_lstm_p3 = None

if 'df' in locals():
    print("🔮 Kahin (LSTM) Eğitiliyor...")
    train_values_c = df['value'].values[:base_train_end]
    
    modelC_p15, modelC_p3, scaler_lstm = train_model_lstm(
        train_values_c, 
        params_p15=best_params_lstm_p15, 
        params_p3=best_params_lstm_p3, 
        scoring_params_p15=PROFIT_SCORING_WEIGHTS, 
        scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3
    )
    
    save_lstm_models(modelC_p15, modelC_p3, scaler_lstm, output_dir='models')
    print("✅ Model C Tamamlandı.")
else:
    print("❌ Veri yok.")


## ⚡ MODEL D: LightGBM (The Flash)

In [ ]:
# 8.1. Model D (LightGBM) - OPTİMİZASYON 🛠️
from jetx_project.optimization import optimize_lightgbm
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

if 'X_a' in locals():
    print("⚡ Şimşek (LightGBM) hız ayarları yapılıyor...")
    
    best_params_lgb_p15 = optimize_lightgbm(X_a, y_p15_a, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS)
    best_params_lgb_p3 = optimize_lightgbm(X_a, y_p3_a, n_trials=15, scoring_params=PROFIT_SCORING_WEIGHTS_P3)
    
    print(f"✅ LightGBM Ayarları Bulundu.")
else:
    print("❌ Veri yok.")


In [ ]:
# 8.2. Model D (LightGBM) - EĞİTİM 🏃
from jetx_project.model_lightgbm import train_model_lightgbm, save_lightgbm_models
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

if 'best_params_lgb_p15' not in locals(): best_params_lgb_p15 = None
if 'best_params_lgb_p3' not in locals(): best_params_lgb_p3 = None

if 'X_a' in locals():
    print("⚡ Şimşek (LightGBM) Eğitiliyor (Özellik Seçimi İle)...")
    
    modelD_p15, modelD_p3, selected_features = train_model_lightgbm(
        X_a, y_p15_a, y_p3_a, 
        params_p15=best_params_lgb_p15, 
        params_p3=best_params_lgb_p3,
        scoring_params_p15=PROFIT_SCORING_WEIGHTS, 
        scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3
    )
    
    print(f"   -> Seçilen En İyi Özellikler: {selected_features}")
    save_lightgbm_models(modelD_p15, modelD_p3, output_dir='models')
    print("✅ Model D Tamamlandı.")
else:
    print("❌ Veri yok.")


## 🧠 MODEL E: MLP (The Brain)

In [ ]:
# 9.1. Model E (MLP) - OPTİMİZASYON 🛠️
from jetx_project.optimization import optimize_mlp
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

if 'X_a' in locals():
    print("🧠 Beyin (MLP) sinir ağlarını yapılandırıyor...")
    
    best_params_mlp_p15 = optimize_mlp(X_a, y_p15_a, n_trials=30, scoring_params=PROFIT_SCORING_WEIGHTS)
    best_params_mlp_p3 = optimize_mlp(X_a, y_p3_a, n_trials=30, scoring_params=PROFIT_SCORING_WEIGHTS_P3)
    
    print(f"✅ MLP Ayarları Bulundu.")
else:
    print("❌ Veri yok.")


In [ ]:
# 9.2. Model E (MLP) - EĞİTİM 🏃
from jetx_project.model_mlp import train_model_mlp, save_mlp_models
from jetx_project.config import PROFIT_SCORING_WEIGHTS, PROFIT_SCORING_WEIGHTS_P3

if 'best_params_mlp_p15' not in locals(): best_params_mlp_p15 = None
if 'best_params_mlp_p3' not in locals(): best_params_mlp_p3 = None

if 'X_a' in locals():
    print("🧠 Beyin (MLP) Eğitiliyor...")
    
    modelE_p15, modelE_p3, mlp_cols = train_model_mlp(
        X_a, y_p15_a, y_p3_a, 
        params_p15=best_params_mlp_p15, 
        params_p3=best_params_mlp_p3,
        scoring_params_p15=PROFIT_SCORING_WEIGHTS, 
        scoring_params_p3=PROFIT_SCORING_WEIGHTS_P3
    )
    
    save_mlp_models(modelE_p15, modelE_p3, mlp_cols, output_dir='models')
    print("✅ Model E Tamamlandı.")
else:
    print("❌ Veri yok.")


## 🤖 MODEL F: Transformer (The Attention)

In [ ]:
# 9.0. Model F (Transformer) - EĞİTİM (Sabit Parametreler) 🤖
# Not: Transformer çok maliyetli olduğu için Optuna optimizasyonu atlanmıştır.
# Standart "Attention" mekanizması ile eğitilir.

from jetx_project.model_transformer import train_model_transformer, save_transformer_models
import os
import sys

# Colab path fix check
project_dir = '/content/newproje'
if os.path.exists(project_dir) and project_dir not in sys.path:
    sys.path.append(project_dir)
if os.path.exists(project_dir):
    %cd /content/newproje

if 'df' in locals() and len(df) > 500:
    print("🤖 Robot (Transformer) Eğitiliyor...")
    
    # Base Train verisi üzerinde eğitim
    train_values_f = df['value'].values[:base_train_end]
    
    # Model Eğitimi (Sabit Ayarlar: 20 Epoch, 4 Heads)
    model_transformer, scaler_transformer = train_model_transformer(train_values_f)
    
    save_transformer_models(model_transformer, scaler_transformer, output_dir='models')
    print("✅ Model F (Transformer) Tamamlandı.")
else:
    print("❌ Yeterli veri yok.")


## 🌊 MODEL G: Fourier Ritim Analizi (The Wave)

In [ ]:
# 9.5. Model G (Fourier) - RİTİM ANALİZİ VE GÖRSELLEŞTİRME 🌊
# Bu model "eğitilmez", matematiksel analiz yapar.
# Burada sinyallerin gücünü görselleştiriyoruz.

from jetx_project.model_fourier import FourierDetector
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

if 'df' in locals():
    print("🌊 Piyasa Ritmi Analiz Ediliyor (Son 2000 el)...")
    
    # Son verileri al
    values = df['value'].values
    
    # Dedektörü Başlat (Çoklu Pencere)
    detector = FourierDetector(window_sizes=[64, 256, 1024])
    
    # Analiz Yap (Batch Mode)
    fourier_df = detector.analyze_batch(values)
    
    # Son 500 eli çizdirelim
    plot_len = 500
    if len(fourier_df) > plot_len:
        subset = fourier_df.iloc[-plot_len:]
        
        plt.figure(figsize=(15, 6))
        
        # 1. Ritim Gücü Grafiği
        plt.subplot(2, 1, 1)
        plt.plot(subset['fourier_strength_64'].values, label='Kısa Vade (Hızlı)', alpha=0.6)
        plt.plot(subset['fourier_strength_256'].values, label='Orta Vade (Ritim)', linewidth=2)
        plt.plot(subset['fourier_strength_1024'].values, label='Uzun Vade (Trend)', linestyle='--')
        plt.title('Piyasa Ritim Gücü (Sinyal Netliği)')
        plt.ylabel('Güç (0-1)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. Faz Grafiği (Döngü Nerede?)
        plt.subplot(2, 1, 2)
        plt.plot(subset['fourier_phase_256'].values, color='purple', label='Döngü Konumu (0=Dip, 1=Tepe)')
        plt.axhline(0.9, color='red', linestyle=':', label='Zirve Bölgesi')
        plt.title('Döngü Konumu (Faz)')
        plt.ylabel('Faz')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Analiz Tamamlandı. Sinyaller Meta-Learner'a gönderilmeye hazır.")
        print(f"Son Durum (Orta Vade): Güç={subset['fourier_strength_256'].iloc[-1]:.2f}, Faz={subset['fourier_phase_256'].iloc[-1]:.2f}")
    else:
        print("Grafik için yeterli veri yok.")
else:
    print("❌ Veri bulunamadı.")


In [ ]:
# 9.5. ENSEMBLE STACKING (Meta-Learner) Eğitimi (Meta Train)
from jetx_project.ensemble import prepare_meta_features, train_meta_learner, save_meta_learner, predict_meta
from jetx_project.model_lstm import create_sequences
# Reloading Imports (Safety Net)
from jetx_project.model_a import load_models as load_model_a
from jetx_project.model_b import load_memory as load_model_b
from jetx_project.model_lstm import load_lstm_models
from jetx_project.model_lightgbm import load_lightgbm_models
from jetx_project.model_mlp import load_mlp_models
from jetx_project.model_transformer import load_transformer_models
from jetx_project.model_hmm import load_hmm_model, predict_categorical_hmm_states

if 'df' in locals() and len(df) > 2000:
    print("--- Ensemble Stacking (Meta-Learner) Eğitiliyor ---")
    
    # --- SAFETY CHECK: RELOAD MODELS IF MISSING ---
    # Colab runtime might have cleared variables. Reload from disk if needed.
    if 'modelA_p15' not in locals():
        print("⚠️ Variables missing. Reloading models from disk...")
        modelA_p15, modelA_p3, modelA_x = load_model_a('models')
        
        nbrs, pca, targets, _ = load_model_b('models') # patterns not needed for prediction
        
        modelC_p15, modelC_p3, scaler_lstm = load_lstm_models('models')
        
        modelD_p15, modelD_p3 = load_lightgbm_models('models')
        
        modelE_p15, modelE_p3, mlp_cols = load_mlp_models('models')
        
        model_transformer, scaler_transformer = load_transformer_models('models')
        
        hmm_model, hmm_map, hmm_bins = load_hmm_model('models')
        # Re-calc hmm_states_mapped if not present
        if 'hmm_states_mapped' not in locals():
             print("Recalculating HMM states...")
             # Use categorical prediction since we loaded categorical model
             # Note: The training cell used causal prediction, but for simple reload we can use standard
             # or simple re-run causal if 'predict_categorical_hmm_states_causal' imported.
             # For simplicity here, let's use the standard predict function or assume it's loaded.
             # Better: just use what we have.
             from jetx_project.model_hmm import predict_categorical_hmm_states_causal
             all_values = df['value'].values
             hmm_states_mapped = predict_categorical_hmm_states_causal(hmm_model, all_values, hmm_map, bins=hmm_bins, window_size=200)
        
        print("✅ Models reloaded successfully.")
    # -----------------------------------------------

    meta_start_idx = base_train_end
    meta_end_idx = meta_train_end
    print(f"Meta Train Aralığı: {meta_start_idx} - {meta_end_idx}")
    # 1. Meta Train Seti İçin Tahminleri Topla
    # Model A (CatBoost)
    from jetx_project.model_a import prepare_model_a_data
    X_meta, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=meta_start_idx)
    X_meta = X_meta[X_meta.index < meta_end_idx]
    # Hedefler
    y_meta_true_15 = (df['value'].values[meta_start_idx+1 : meta_end_idx+1] >= 1.5).astype(int)
    # Uzunluk Kontrolü
    min_len = min(len(X_meta), len(y_meta_true_15))
    X_meta = X_meta.iloc[:min_len]
    y_meta_true_15 = y_meta_true_15[:min_len]
    print(f"Meta Train Örnek Sayısı: {min_len}")
    preds_a_meta = modelA_p15.predict_proba(X_meta)[:, 1]
    # Model B (k-NN)
    preds_b_meta = []
    from jetx_project.model_b import create_pattern_vector, predict_model_b
    pca_obj = locals().get('pca', None) 
    for idx in X_meta.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_meta.append(p15)
        else:
            preds_b_meta.append(0.5)
    preds_b_meta = np.array(preds_b_meta)
    # Model C (LSTM)
    seq_len = 200
    lstm_start_offset = X_meta.index[0] - seq_len
    lstm_end_offset = X_meta.index[-1]
    meta_values_extended = df['value'].values[lstm_start_offset : lstm_end_offset + 1]
    meta_values_scaled = scaler_lstm.transform(meta_values_extended.reshape(-1, 1))
    X_lstm_meta, _, _, _ = create_sequences(meta_values_scaled, meta_values_extended, seq_len)
    preds_c_meta = modelC_p15.predict(X_lstm_meta).flatten()
    # Model D (LightGBM)
    preds_d_meta = modelD_p15.predict_proba(X_meta)[:, 1]
    # Model E (MLP)
    X_meta_mlp = X_meta[mlp_cols]
    preds_e_meta = modelE_p15.predict_proba(X_meta_mlp)[:, 1]
    # Model F (Transformer)
    meta_values_scaled_trans = scaler_transformer.transform(meta_values_extended.reshape(-1, 1))
    X_trans_meta, _, _, _ = create_sequences(meta_values_scaled_trans, meta_values_extended, seq_len)
    preds_f_meta = model_transformer.predict(X_trans_meta)
    preds_f_meta_p15 = preds_f_meta[0].flatten()
    # HMM States
    hmm_meta = hmm_states_mapped[X_meta.index]
    # 2. Meta-Features Hazırla
    final_len = min(len(preds_a_meta), len(preds_b_meta), len(preds_c_meta), len(preds_d_meta), len(preds_e_meta), len(preds_f_meta_p15))
    meta_values_raw = df['value'].values[meta_start_idx : meta_end_idx]
    meta_values_raw = meta_values_raw[:final_len]
    meta_X_train = prepare_meta_features(
        preds_a_meta[:final_len], 
        preds_b_meta[:final_len], 
        preds_c_meta[:final_len], 
        preds_d_meta[:final_len], 
        preds_e_meta[:final_len], 
        hmm_meta[:final_len],
        values=meta_values_raw,
        preds_transformer=preds_f_meta_p15[:final_len]
    )
    y_meta_train = y_meta_true_15[:final_len]
    # 3. Eğit
    meta_model, meta_scaler = train_meta_learner(meta_X_train, y_meta_train)
    save_meta_learner(meta_model, meta_scaler, output_dir='models')
    print("Meta-Learner eğitildi ve kaydedildi.")
else:
    print("Yeterli veri yok.")


In [ ]:
# 9.8. RL Agent (The Strategist) Eğitimi
from jetx_project.model_rl import train_rl_agent, save_rl_agent

if 'meta_model' in locals():
    print("RL Agent (The Strategist) eğitiliyor...")
    
    # Meta-Learner tahminlerini alalım
    meta_probs = predict_meta(meta_model, meta_scaler, meta_X_train)
    
    # DataFrame oluştur
    rl_train_df = pd.DataFrame({
        'value': meta_values_raw, # Gerçek değerler
        'prob_1.5': meta_probs,
        'hmm_state': hmm_meta[:final_len] # HMM durumları
    })
    
    rl_agent = train_rl_agent(rl_train_df)
    save_rl_agent(rl_agent, output_dir='models')
    print("RL Agent tamamlandı.")
else:
    print("Meta-Learner hazır değil.")


In [ ]:
# 10. BÜYÜK FİNAL: Simülasyon (Test Seti Üzerinde)
# Strateji Güncellemesi: Sadece Eşik 0.70 (HMM Filtresi Yok)

print("--- BÜYÜK FİNAL: Simülasyon Başlıyor (Test Seti) ---")

test_start_idx = meta_train_end
test_end_idx = len(df) - 1
test_len = test_end_idx - test_start_idx
print(f"Test Seti Başlangıç: {test_start_idx}, Uzunluk: {test_len}")

if test_len > 200:
    # Test verisini hazırla
    sim_df = df.iloc[test_start_idx:test_end_idx].copy()
    sim_df = sim_df.reset_index(drop=True)
    
    print("Modeller tahmin ediyor...")
    
    # 1. Toplu Tahmin (Batch Prediction)
    
    # Model A (CatBoost)
    from jetx_project.model_a import prepare_model_a_data
    X_test_a, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=test_start_idx)
    min_len = min(len(X_test_a), len(sim_df))
    X_test_a = X_test_a.iloc[:min_len]
    sim_df = sim_df.iloc[:min_len]
    
    sim_df['prob_A_1.5'] = modelA_p15.predict_proba(X_test_a)[:, 1]
    
    # Model B (k-NN) - VECTORIZED / BATCH
    print("Model B (k-NN) Batch Prediction...")
    from jetx_project.model_b import create_pattern_vector, predict_model_b
    pca_obj = locals().get('pca', None)
    
    patterns_list = []
    valid_indices = []
    for idx in X_test_a.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            patterns_list.append(pat)
            valid_indices.append(len(patterns_list)-1)
        else:
            patterns_list.append(np.zeros(300))
            
    import numpy as np
    X_patterns = np.array(patterns_list)
    preds_b_p15, _, _ = predict_model_b(nbrs, pca_obj, targets, X_patterns)
    sim_df['prob_B_1.5'] = preds_b_p15
    
    # Model C (LSTM)
    seq_len = 200
    lstm_start = X_test_a.index[0] - seq_len
    test_values_extended = df['value'].values[lstm_start : X_test_a.index[-1] + 1]
    test_values_scaled = scaler_lstm.transform(test_values_extended.reshape(-1, 1))
    X_lstm, _, _, _ = create_sequences(test_values_scaled, test_values_extended, seq_len)
    sim_df['prob_C_1.5'] = modelC_p15.predict(X_lstm, verbose=0).flatten()[:min_len]
    
    # Model D (LightGBM)
    sim_df['prob_D_1.5'] = modelD_p15.predict_proba(X_test_a)[:, 1]
    
    # Model E (MLP)
    X_test_mlp = X_test_a[mlp_cols]
    sim_df['prob_E_1.5'] = modelE_p15.predict_proba(X_test_mlp)[:, 1]

    # Model F (Transformer)
    meta_values_scaled_trans = scaler_transformer.transform(test_values_extended.reshape(-1, 1))
    X_trans_test, _, _, _ = create_sequences(meta_values_scaled_trans, test_values_extended, seq_len)
    preds_f_test = model_transformer.predict(X_trans_test, verbose=0)
    sim_df['prob_F_1.5'] = preds_f_test[0].flatten()[:min_len]
    
    # HMM Feature for Filter (Gözlem için tutuyoruz ama şart değil)
    hmm_test = hmm_states_mapped[X_test_a.index]
    sim_df['hmm_state'] = hmm_test
    
    # 2. Meta-Learner Tahmini (+ Fourier G)
    print("   -> Model G (Fourier) dahil ediliyor...")
    from jetx_project.ensemble import prepare_meta_features, predict_meta
    
    meta_X_test = prepare_meta_features(
        sim_df['prob_A_1.5'].values,
        sim_df['prob_B_1.5'].values,
        sim_df['prob_C_1.5'].values,
        sim_df['prob_D_1.5'].values,
        sim_df['prob_E_1.5'].values,
        hmm_test,
        values=sim_df['value'].values,
        preds_transformer=sim_df['prob_F_1.5'].values
    )
    
    sim_df['meta_prob'] = meta_model.predict_proba(meta_X_test)[:, 1]
    
    # 3. Simülasyon
    initial_balance = 1000
    balance = initial_balance
    balance_history = [balance]
    bet_amount = 10
    
    wins = 0
    losses = 0
    
    print("\n--- OPTİMİZE EDİLMİŞ Simülasyon Sonuçları ---")
    print("Kurallar: Güven > 0.70 (HMM Filtresi YOK)")
    
    for i, row in sim_df.iterrows():
        prob = row['meta_prob']
        real_outcome = row['value']
        
        # Strateji: Sadece Prob > 0.70
        if prob > 0.70:
            if real_outcome >= 1.50:
                profit = bet_amount * 0.50
                balance += profit
                wins += 1
            else:
                balance -= bet_amount
                losses += 1
        
        balance_history.append(balance)
        
    print(f"Başlangıç Kasa: {initial_balance}")
    print(f"Bitiş Kasa: {balance:.2f}")
    print(f"Kar/Zarar: {balance - initial_balance:.2f}")
    print(f"Toplam Bahis: {wins + losses}")
    print(f"Kazanma Oranı: {wins / (wins + losses) if (wins+losses) > 0 else 0:.2%}")
    
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6))
    plt.plot(balance_history)
    plt.title('Simülasyon Kasa Grafiği (Eşik 0.70)')
    plt.xlabel('Oyun Sayısı')
    plt.ylabel('Bakiye')
    plt.axhline(y=initial_balance, color='r', linestyle='--')
    plt.show()
    
else:
    print("Test seti çok kısa.")


In [ ]:
# 11. Modelleri Yedekle (Google Drive & Local)
import os
import datetime
import shutil
from google.colab import drive, files

print("--- Modeller Yedekleniyor ---")

# 1. Zip Oluştur
print("Modeller sıkıştırılıyor (models.zip)...")
!zip -r models.zip models/

# 2. Google Drive Yedekleme
try:
    print("Google Drive bağlanıyor...")
    drive.mount('/content/drive')
    
    # Zaman damgalı klasör adı
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    backup_folder_name = f"JetX_Backup_{timestamp}"
    backup_path = f"/content/drive/MyDrive/{backup_folder_name}"
    
    if not os.path.exists(backup_path):
        os.makedirs(backup_path)
        print(f"Yedekleme klasörü oluşturuldu: {backup_path}")
        
    # Kopyala
    shutil.copy("models.zip", f"{backup_path}/models.zip")
    print(f"✅ BAŞARILI: Modeller Drive'a yedeklendi -> {backup_path}/models.zip")
    
except Exception as e:
    print(f"⚠️ UYARI: Google Drive yedeklemesi başarısız oldu: {e}")
    print("Yerel indirme deneniyor...")

# 3. Yerel İndirme (Her durumda sunulur)
try:
    files.download('models.zip')
    print("✅ İndirme başlatıldı.")
except Exception as e:
    print(f"İndirme hatası: {e}")
